In [47]:
import pandas as pd
import numpy as np
import networkx as nx

In [7]:
user = pd.read_json('user.json')
user.head(5)

,ID,tweet,profile
0,15750898,[A family fears they may have been cheated out...,"{'id': 15750898, 'id_str': '15750898', 'name':..."
1,1659167666,[RT @VonteThePlug: Yeah but he ain’t got one h...,"{'id': 1659167666, 'id_str': '1659167666', 'na..."
2,34743251,"[RT @elonmusk: Aloha, welcome back from space ...","{'id': 34743251, 'id_str': '34743251', 'name':..."
3,207809313,[एक राष्ट्र - एक राशन कार्ड की ओर अग्रसर भारत\...,"{'id': 207809313, 'id_str': '207809313', 'name..."
4,24212655,[RT @dominiquecrenn: So much negativity and go...,"{'id': 24212655, 'id_str': '24212655', 'name':..."


In [18]:
### Part 1 选择需要分析的individual trait
location = []
for i in range(len(user)):
    if user['profile'][i]==None:
        location.append('Unknown')
        continue
    location.append(user['profile'][i]['location'])

In [20]:
user['location'] = location
user.head(5)

,ID,tweet,profile,location
0,15750898,[A family fears they may have been cheated out...,"{'id': 15750898, 'id_str': '15750898', 'name':...","Tampa, FL"
1,1659167666,[RT @VonteThePlug: Yeah but he ain’t got one h...,"{'id': 1659167666, 'id_str': '1659167666', 'na...","Jacksonville Beach, FL"
2,34743251,"[RT @elonmusk: Aloha, welcome back from space ...","{'id': 34743251, 'id_str': '34743251', 'name':...","Hawthorne, CA"
3,207809313,[एक राष्ट्र - एक राशन कार्ड की ओर अग्रसर भारत\...,"{'id': 207809313, 'id_str': '207809313', 'name...","6-A, Deen Dayal Upadhyay Marg,"
4,24212655,[RT @dominiquecrenn: So much negativity and go...,"{'id': 24212655, 'id_str': '24212655', 'name':...","Chicago, Illinois"


In [24]:
trait = user[['ID', 'location']]
bot_label = pd.read_csv('train.csv', sep='\t')
user_with_label = pd.merge(trait, bot_label, on='ID', how='inner')
user_with_label['Label'].value_counts()

0    6908
1    1829
Name: Label, dtype: int64

In [27]:
edge = pd.read_json('edge.json')
edge.head()

,seed_user_id,relation,relation_user_id
0,15750898,followers,2324715174
1,15750898,friends,1279902697991409664
2,1659167666,followers,329721192
3,1659167666,followers,396721750
4,1659167666,friends,243412850


In [39]:
# 处理成(2, E)的数据，其中第0行节点follow第1行节点
followers = edge[edge['relation']=='followers']
friends = edge[edge['relation']=='friends']

In [44]:
edges_1 = np.array([followers['relation_user_id'].to_list(),followers['seed_user_id'].to_list()])
edges_2 = np.array([friends['seed_user_id'].to_list(),friends['relation_user_id'].to_list()])
edges = np.concatenate([edges_1, edges_2], axis=1)

In [111]:
a = list(edges[0])
b = list(edges[1])
c = user_with_label['ID'].to_list()
followers_label = set(a)&set(c)
followed_label = set(b)&set(c)

In [112]:
followers_label&followed_label
## TODO: 这个数据集tie两端都有身份label的只有86个

{1976841,
 14594813,
 14643178,
 15008142,
 15956067,
 16953652,
 18565087,
 19026735,
 20264905,
 20349330,
 21651982,
 23553743,
 24198971,
 25154114,
 34156176,
 34716038,
 35100670,
 36658407,
 39651447,
 40725502,
 46518803,
 48074464,
 54265553,
 55815840,
 64576418,
 65776012,
 76710013,
 77808373,
 86601594,
 87070479,
 87154044,
 89788329,
 94847504,
 107797719,
 114425330,
 119152527,
 119533988,
 127361253,
 156005263,
 169703787,
 176348732,
 177758370,
 209001970,
 214134167,
 222286198,
 253006900,
 258620586,
 268482830,
 287938981,
 289932443,
 296020190,
 306510658,
 308006744,
 323138766,
 326913391,
 327077249,
 340803178,
 400624214,
 408673732,
 422218178,
 428605324,
 432899489,
 453495862,
 476562084,
 489979871,
 576497349,
 580224794,
 591909448,
 597847340,
 603660185,
 607710343,
 621020931,
 853100948,
 913468807,
 1082868295,
 1124924107,
 1137559482,
 1264286744,
 1342859496,
 1702605908,
 1720554090,
 1878871010,
 1901298962,
 1920805208,
 2181042705,
 21

In [68]:
# 创建一个空的NetworkX图
G = nx.DiGraph()

# 添加节点及其属性
for index, row in user_with_label.iterrows():
    G.add_node(row['ID'], feature=row['location'], label=row['Label'])

# 添加边
for e in range(len(edges[0])):
    G.add_edge(edges[0][e], edges[1][e])

print('Num Nodes:', len(G.nodes))
print('Num Edges:', len(G.edges))

Num Nodes: 24696
Num Edges: 17792


In [82]:
import matplotlib.pyplot as plt
# 设置节点颜色
node_colors = {node: user_with_label[user_with_label['ID'] == node]['Label'].item() for node in G.nodes}
# 绘制图
pos = nx.random_layout(G)
nx.draw(G, pos, with_labels=True, node_color=[node_colors[node] for node in G.nodes], cmap=plt.cm.tab10)
plt.show()

ValueError: can only convert an array of size 1 to a Python scalar